# `Step 2: Text Embeddings`

# 0- `Config`

In [ ]:
# Path Management
import pickle
import os
# Data Handling 
import pandas as pd
import numpy as np
# Transformers 
from transformers import BertModel, BertTokenizer
# Tokenization
import torch
from tqdm import tqdm

In [ ]:
try:
    if manager == 1:
        print("Jupyter has been running from Manager") 
except:
    temp  = "temp"
    %run s0_config.ipynb 

# 1- `Load`

In [ ]:
import_name = "subject_data"
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_setup}", f"{import_name}" + ".pkl")) , 'rb') as file:
    subject_data = pickle.load(file)   

## `Overview`

In [ ]:
#subject_data = subject_data[:200]
subject_data

In [ ]:
subject_data = subject_data.iloc[:100]

# 2- `Operation`

### 1-) `Context Construction`

In [ ]:
texts = subject_data['Cell_Type'] + " " + subject_data['Gene_Marker']
texts

### 2-) `Transformer Selection`

In [ ]:
# Step 1: Load pre-trained model tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Step 2: Tokenization 
tokenized = texts.apply(lambda x: tokenizer(x, padding='max_length', truncation=True, max_length=512, return_tensors="pt")) 
# Step 3: Model Selection  
model = BertModel.from_pretrained('bert-base-uncased')

### 3-) `Tokenization`

In [ ]:

batch_size = 32
Combined_Text_Embeddings = []

# Assuming model is already loaded and set to evaluation mode with model.eval()

for i in tqdm(range(0, len(tokenized), batch_size)):
    # Here, make sure you're extracting batches correctly
    batch = tokenized[i:i+batch_size]
    
    # Now, prepare your inputs. This assumes each entry in `batch` is a dictionary
    # returned by the tokenizer corresponding to a single example.
    batch_input_ids = torch.stack([item['input_ids'] for item in batch])
    batch_attention_mask = torch.stack([item['attention_mask'] for item in batch])
    
    # Ensure dimensions are correct: [batch_size, sequence_length]
    # If you encounter an error here, check the structure and content of `batch`
    inputs = {
        'input_ids': batch_input_ids.squeeze(),  # Remove any extra dimensions
        'attention_mask': batch_attention_mask.squeeze()  # Remove any extra dimensions
    }
    
    with torch.no_grad():
        outputs = model(**inputs)
        batch_embeddings = outputs.pooler_output
        Combined_Text_Embeddings.append(batch_embeddings)

# Combine embeddings from all batches
Embeddings_BERT = torch.cat(Combined_Text_Embeddings, dim=0)

#Embeddings_BERT = 1 

In [ ]:
Embeddings_BERT.shape

In [ ]:
len(data_full["Embeddings"].iloc[0]) 

### 4-) `Main Data Creation`

In [ ]:
# Step 1 
data_full = subject_data.copy() 
# Step 2: 
#data_full["Cell_Type_Embeddings"] = Embeddings_BERT  
data_full["Embeddings"] = [embedding.tolist() for embedding in Embeddings_BERT]

In [ ]:
#embeddings_list = data_full["Cell_Type_Embeddings"].tolist()  # Get the column as a list of lists
#embeddings_tensor = torch.tensor(embeddings_list)             # Convert to a tensor 

# 3- `End`

In [ ]:
export_name = "data_full" 
with open(os.path.join(access_data_path(f"{process_barcode}/data/{folder_text}", f"{export_name}" + ".pkl"))  , 'wb') as file: 
    pickle.dump(data_full, file)   